# 1Hive Economy  

This model aims to create a simulation of the 1hive economy, it assumes that 1hive can capture fee inflows by directing outflows towards productive activities and represents this growth potential as a logistic curve that represents market saturation. We assume that outflows will result in increased market saturation, and that participants will increase outflows when the perceived opportunity for growth is higher, and reduce outflows as the opportunity for growth decreases. 
    
For the purpose of this model we differentiate the honey supply into two buckets, the common pool and the circulating supply. We model inflows to the system (in stable value terms) as a function of saturation and market size, and have saturation decay over time. We define a conversion price that adjusts each timestep as a ratio of market cap to inflows divided by saturation, and use this price to model inflows as a movement of honey from the circulating supply into the common pool. 

We set an outflow boundary from the common pool as an abstraction of how conviction voting limits the **maximum possible outflow rate from a pool of funds given a certain parameterization of conviction voting**, though in practice the real outflow rate would tend to be much lower than this boundary due to the ability for participants to support signaling proposals (eg abstain), or for proposal to recieve some support but ultimately fail to reach sufficient support to execute. To reflect the expected behavior of participants to modulate outflows based on their perception of the growth potential we make the actual outflow rate adjust based on level of saturation. 

We are interested in exploring parameter choices for internal mechanisms and analzying the sensitivity to parameters of environmental processes to better understand how the protocol will respond in terms of price, supply, and common pool reserves across various stages of growth. 




In [1]:
import random as rand
import numpy as np 

In [2]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
genesis_states = {
    'reserve': 7473, # Current common pool balance
    'circulation': 18765, # Total honey supply minus common pool balance (26238 - 7473)
    'size': 5318016, # Total market size in terms of fiat inflows per month, as a starting place I took the 24 hour volume for uniswap multiplied by 30, then by 0.003, and then divided by 6   
    'saturation': 0.0, # Percent of market captured by 1hive
    'production': 0.0, # Production state accounting for outflows and upkeep
    'utility': 0.0, # representing diminishing marginal returns on production
    'price': 1, # determined each timestep based on size, saturation, and circulation based on a target P:E ratio parameter
    'netflow': 0.0,
    'adjustment': 0.0
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Model Params
We define some parameters which can be used to tune the behavior of issuance and distribution in the model. Paramters assume that each time step of the model relfect 1 month of real time, the simulation will run for 120 timesteps giving the model as a whole a 10 year time horizon.  

In [3]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
params = {
    'throttle': [0.008], # maximimum proportion of the total supply that can be adjusted in each timestep 
    'outflow_boundary': [0.05], # max proportion of common pool funds that can be spent each timestep 
    'target_reserve_ratio': [0.2], # target ratio of common pool funds to total supply
    'valuation_ratio': [2], # ratio of price to inflows (monthly) when at market saturation 
    'productivity': [1.5],  # scalar parameter to determine impact of outflows on market saturation
    'maintainence': [0.1], # scalar parameter to determine magnitude of saturation decays each time step
    'growth': [0] # rate at which the market size will grow each time step 
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Timestep
We assume that each timestep in cadcad represents 1 month in real time, scaling model parameters to reflect that timescale. 

# Policies and State Update Functions

We define a supply policy and four state update fuctions. 


In [4]:
def supply_policy(params, step, sH, s):

    # first we calculate outlflow as a function of the reserve, outflow boundary, and saturation
    outflow = s['reserve'] * params['outflow_boundary'] / (s['saturation'] + 1)
    
    # then we calculate inflow as a function size, saturation, price
    inflow = s['size'] * s['saturation'] / s['price']

    netflow = inflow - outflow

    # then we calculate current state
    reserve = s['reserve'] + netflow 
    circulation = s['circulation'] - netflow
    supply = reserve + circulation
    ratio = reserve / supply
    

    # Proportional control https://en.wikipedia.org/wiki/Proportional_control
    # corrections are made proportionally to the difference between the target and the current value

    # e = (params['target_reserve_ratio'] - ratio) / 12
    e = (1 - ratio / params['target_reserve_ratio'])/12

    if params['throttle'] != 0: 
        # Corrections bounded by a maximum issuance rate parameter 
        if e < 0:
            adjustment = max(e, -params['throttle']) * supply
        else:
            adjustment = min(e, params['throttle']) * supply 
    else:
        # Corrections are unbounded, issuance is bounded by the maximum outflow rate 
        adjustment = e * supply 


    return ({'netflow':netflow, 'adjustment':adjustment})


def saturation_process(params, step, sH, s):

    # outflows are a function of outflow_boundary parameter and current saturation
    # as saturation approaches 1, outflow_boundary is reduced by half. 
    outflow = s['reserve'] * params['outflow_boundary'] / (1 + s['saturation'] ** 2 )

    
    # production reduced by the maintainance parameter and then incread by the impact of outflows, which depends on the productivity paramater and the current price. 
    production = s['production'] * (1 - params['maintainence']) + outflow * params['productivity'] * s['price']

    # Utility represents the diminishing returns to production and is bounded at 10, we use the size value to shape the curve because it relates to the maximum inflows and steady state outflows. 
    utility = 10 * production / ( production + s['size'] )

    # saturation modeled as a logistic function, shifted by 6 so that a 0 utility means near 0 saturation, a utility of 5 is around the inflection point, and 10 is near 1. 
    saturation = 1 / (1 + np.exp(-utility + 6))

    # size is a function of current size and growth rate
    size = s['size'] * (1 + params['growth'])


    return ({ 'saturation':saturation, 'size':size, 'utility':utility, 'production':production})


def update_reserve(params, step, sH, s, _input):
    key = 'reserve'
    value = s['reserve'] + _input['netflow'] + _input['adjustment']
    return (key, value)

def update_utility(params, step, sH, s, _input):
    key = 'utility'
    value = _input['utility']
    return (key, value)

def update_production(params, step, sH, s, _input):
    key = 'production'
    value = _input['production']
    return (key, value)

def update_circulation(params, step, sH, s, _input):
    key = 'circulation'
    value = s['circulation'] - _input['netflow']
    return (key, value)

def update_size(params, step, sH, s, _input):
    key = 'size'
    value =  _input['size']
    return (key, value)

def update_saturation(params, step, sH, s, _input):
    key = 'saturation'
    value =  _input['saturation']
    return (key, value)

def update_price(params, step, sH, s, _input):
    key = 'price'
    # price is function of size, saturation, valuation ratio, and total supply and circulating supply

    # base token value is size of market divided by total supply 
    market_potential = s['size'] / (s['circulation'] + s['reserve'])

    # valuation ratio decreases as system reaches market saturation 
    valuation_ratio = params['valuation_ratio'] / max(0.5, s['saturation'])
    
    # Price increases exponentially as ratio of circulating supply to total supply decreases
    supply_sensitivity = 1 / (s['circulation'] / (s['circulation'] + s['reserve']) ) ** 2 

    value = market_potential * valuation_multiple * supply_sensitivity

    return (key, value)

def update_netflow(params, step, sH, s, _input):
    key = 'netflow'
    value =  _input['netflow']
    return (key, value)

def update_adjustment(params, step, sH, s, _input):
    key = 'adjustment'
    value =  _input['adjustment']
    return (key, value)

# Partial State Update Blocks


In [5]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, the user specifies if state update functions will be run in series or in parallel
partial_state_update_blocks = [
    { 
        'policies': {
            'supply_policy': supply_policy,
            'saturation_process': saturation_process

        },
        'variables': { # The following state variables will be updated simultaneously
            'reserve': update_reserve,
            'circulation': update_circulation,
            'netflow': update_netflow,
            'adjustment': update_adjustment,
            'price': update_price,
            'saturation': update_saturation,
            'utility': update_utility,
            'size': update_size,
            'production': update_production
        }
    }
]
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Simulation Configuration Parameters
Lastly, we define the number of timesteps and the number of Monte Carlo runs of the simulation. These parameters must be passed in a dictionary, in `dict_keys` `T` and `N`, respectively. In our example, we'll run the simulation for 10 timesteps. And because we are dealing with a deterministic system, it makes no sense to have multiple Monte Carlo runs, so we set `N=1`. We'll ignore the `M` key for now and set it to an empty `dict`

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's omit it
sim_config_dict = {
    'T': range(120),
    'N': 1,
    'M': params
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Putting it all together
We have defined the state variables of our system and their initial conditions, as well as the state update functions, which have been grouped in a single state update block. We have also specified the parameters of the simulation (number of timesteps and runs). We are now ready to put all those pieces together in a `Configuration` object.

In [7]:
#imported some addition utilities to help with configuration set-up
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

exp = Experiment()
c = config_sim(sim_config_dict)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
del configs[:]
exp.append_configs(initial_state=genesis_states, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_configs=c #preprocessed dictionaries containing simulation parameters
                      )

# Running the engine
We are now ready to run the engine with the configuration defined above. Instantiate an ExecutionMode, an ExecutionContext and an Executor objects, passing the Configuration object to the latter. Then run the `execute()` method of the Executor object, which returns the results of the experiment in the first element of a tuple.

In [8]:
%%capture
from cadCAD.engine import ExecutionMode, ExecutionContext
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

from cadCAD.engine import Executor

simulation = Executor(exec_context=local_mode_ctx, configs=configs) # Pass the configuration object inside an array
raw_system_events, tensor_field, sessions = simulation.execute() # The `execute()` method returns a tuple; its first elements contains the raw results


# Analyzing the results
We can now convert the raw results into a DataFrame for analysis

In [9]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result['total_supply'] = simulation_result['reserve'] + simulation_result['circulation']
simulation_result['market_cap'] = simulation_result['total_supply'] * simulation_result['price']
simulation_result['ratio'] = simulation_result['reserve'] / simulation_result['total_supply']
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

reserve   circulation     size  saturation  \
subset run timestep substep                                                   
0      1   0        0        7473.000000  18765.000000  5318016    0.000000   
           1        1        6889.446000  19138.650000  5318016    0.002475   
           2        1        6345.904114  19473.967118  5318016    0.009334   
           3        1        5857.821505  19755.490757  5318016    0.020846   
           4        1        5442.548384  19965.857380  5318016    0.034898   
...                                  ...           ...      ...         ...   
           116      1        8377.902833   2294.931929  5318016    0.964868   
           117      1        8310.883298   2276.568786  5318016    0.964869   
           118      1        8244.399543   2258.352925  5318016    0.964870   
           119      1        8178.447319   2240.283129  5318016    0.964871   
           120      1        8113.022410   2222.358195  5318016    0.964871   

                               production   utility         price     netflow  \
subset run timestep substep                                                     
0      1   0        0        0.000000e+00  0.000000      1.000000    0.000000   
           1        1        5.604750e+02  0.001054   1585.050930 -373.650000   
           2        1        8.195086e+05  1.335243   1511.574059 -335.317118   
           3        1        1.456918e+06  2.150453   1448.287687 -281.523639   
           4        1        1.947236e+06  2.680204   1396.046004 -210.366624   
...                                   ...       ...           ...         ...   
           116      1        7.207852e+07  9.312887  22159.621313   18.511656   
           117      1        7.208146e+07  9.312913  22338.385243   18.363143   
           118      1        7.208413e+07  9.312937  22518.585792   18.215861   
           119      1        7.208654e+07  9.312958  22700.234989   18.069796   
           120      1        7.208873e+07  9.312978  22883.344917   17.924934   

                             adjustment  simulation  total_supply  \
subset run timestep substep                                         
0      1   0        0          0.000000           0  26238.000000   
           1        1       -209.904000           0  26028.096000   
           2        1       -208.224768           0  25819.871232   
           3        1       -206.558970           0  25613.312262   
           4        1       -204.906498           0  25408.405764   
...                                 ...         ...           ...   
           116      1        -86.071248           0  10672.834763   
           117      1        -85.382678           0  10587.452084   
           118      1        -84.699617           0  10502.752468   
           119      1        -84.022020           0  10418.730448   
           120      1        -83.349844           0  10335.380604   

                               market_cap     ratio  
subset run timestep substep                          
0      1   0        0        2.623800e+04  0.284816  
           1        1        4.125586e+07  0.264693  
           2        1        3.902865e+07  0.245776  
           3        1        3.709544e+07  0.228702  
           4        1        3.547130e+07  0.214203  
...                                   ...       ...  
           116      1        2.365060e+08  0.784974  
           117      1        2.365066e+08  0.784975  
           118      1        2.365071e+08  0.784975  
           119      1        2.365076e+08  0.784975  
           120      1        2.365081e+08  0.784976  

[121 rows x 13 columns]

# Analysis 

We have 4 different subsets of data based on different parameters. 

* subset 0: 20% reserve target, 0.05 outlfow rate from common pool per month, no throttle, no inflows
* subset 1: 20% reserve target, 0.05 outflow rate from common pool per month, 0.004 per month throttle, no inflows
* subset 2: 20% reserve target, 0.025 outflow rate from common pool per month, no throttle, no inflows
* subset 3: 20% reserve target, 0.05 outflow rate from the common pool per month, no throttle, and 0.02 inflows

I used a 20% reserve target and 0.05 maxiumum outflow rate as a baseline. Even with a 20% reserve target, the outflow rate and lack of inflows result in a equilibrium reserve of around 18%, if we conservatively assume the maximum outflow rate of 0.05 of the common pool, we end up with a max of ~10.8% issuance per year. 

In the base case we do not specify the issuance rate, we only specify the target and conviction voting parameters that control the maximum outflow. However, if we want to define a maximum issuance rate directly we can use the throttle paramter. In subset 1, this is set to 0.004 or 4.8% per year, resulting in a reserve that stabilizes around 7.5%. It's worth noting that separting the max issuance rate from the conviction parameters, we can potentially remove governance over the max issuance (throttle), while retaining the ability to adjust conviction parameterization as needed, and we can more easily explain the maximum issuance rate without having to explain conviction voting parameterization in-depth. 

In practice we would expect the actual outflows to be much lower than theoretical maximum allows by a given set of conviction parameters. In subset 2, I turned the throttle back off but reduced the outflow rate by half. Reducing outflow rate results in slower overall supply growth, and the reserve stabilizing at a bit higher. From a system perspective, this means that if Honey holders choose to stake honey on the abstain proposals they would effectively be voting to reduce both the spending and the issuance rate. This ensures that honey holders do not have an incentive to overspend, because they always have the option to abstain and reduce issuance and outflows if they do not think any of the current proposals provide positive expected future value. 

Finally, in subset 4, I've set inflows to be greater than outflows, showing how over time this results in the reserves increasing beyond the target, and the supply of Honey decreasing as honey is burned from the common pool. Inflows are specified as a fixed ratio of the circulating supply in the model, which is completely arbitrary, but does help illustrate how consistent net inflows would impact both the total supply and reserves. Total supply would become deflationary, while the reserve would stabilize above the target reserve ratio. 






In [10]:



import plotly.express as px
fig = px.line(
    simulation_result,
    x='timestep',
    y=['total_supply', 'circulation', 'reserve'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [11]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['netflow', 'adjustment'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [12]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['ratio', 'saturation'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [13]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['price'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [14]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['market_cap', 'size'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [15]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['utility'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()